<a href="https://colab.research.google.com/github/hudada369/fy2020-repo-config/blob/master/Pytorch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **模型：模型的创建步骤和nn.Module细节学习**

**搭建模型的容器Containers 包括 nn.Sequential nn.ModuleList, nn.ModuleDict**


1.   Pytorch模型的建立
2.   Pytorch的容器
1.   AlexNet网络的构建
2.   总结回顾







模型的内容：nn.Module

1. 模型创建
  1. 构建网络层：卷积层 池化层 激活函数层
  2. 拼接网络层：Lenet AlexNet ResNet等
2. 权值初始化：
  1. Xavier Kaiming 均匀分布 正态分布

通过对Lenet网络的分析，可以得到模型的两大要素
1. 构建子模块 例如卷积网络中卷积层 池化层 全连接层
2. 拼接子模块（有了子模块，我们把子模块按照一定的顺序，逻辑进行拼接起来得到最终的 LeNet 模型）

进行代码的分析，nnModule有Lenet模型，对其查看源码：
1. lenet.py 文件，在这里面有个 「LeNet 类，继承了 nn.Module」。并且我们发现在「它的__init__方法里面实现了各个子模块的构建」
2. 子模块的拼接是在 forward 里面

在模型的概念当中，我们有一个非常重要的概念叫做nn.Module,  我们所有的模型，所有的网络层都是继承于这个类的。
class Net(nn.Module):
  函数

nn.Module类 包含在 torch.nn中
torch.nn:
1. nn.Parameter 张量子类，表示可学习参数，如Weights bias
2. nn.Module 所有网络层基类，管理网络属性
  1. 属性
   1. self._parameters  = OrderedDict() 存储管理属于 nn.Parameter 类的属性，例如权值，偏置这些参数
   2. self._buffers= OrderedDict()  存储管理缓冲属性，如 BN 层中的 running_mean，std 等都会存在这里面
   3. self._backward_hooks= OrderedDict() 存储管理钩子函数（5 个与 hooks 有关的字典
   4. self._forward_pre_hooks= OrderedDict()
   5. self._forward_hooks= OrderedDict()
   6. self._state_dict_hooks= OrderedDict()
   7. self._load_state_dict_pre_hooks= OrderedDict()
   8. self._modules = OrderedDict() 存储管理 nn.Module 类， 比如 LeNet 中，会构建子模块，卷积层，池化层，就会存储在 modules 中
3. nn.functional 函数具体实现，如卷积 赤化 激活函数
4. nn.init 


nn.Module创建子模块流程：
 先有一个大的 Module 继承 nn.Module 这个基类，比如上面的 LeNet，然后这个大的 Module 里面又可以有很多的子模块，这些子模块同样也是继承于 nn.Module， 在这些 Module 的__init__方法中，会先通过调用父类的初始化方法进行 8 个属性的一个初始化。然后在构建每个子模块的时候，其实分为两步，第一步是初始化，然后被__setattr__这个方法通过判断 value 的类型将其保存到相应的属性（是模型还是参数）字典里面去，然后再进行赋值给相应的成员。这样一个个的构建子模块，最终把整个大的 Module 构建完毕

2. nn.Module 进行总结：

 1. 一个 module 可以包含多个子 module（LeNet 包含卷积层，池化层，全连接层）
 2. 一个 module 相当于一个运算， 必须实现 forward() 函数（从计算图的角度去理解）
 3. 每个 module 都有 8 个字典管理它的属性（最常用的就是_parameters，_modules ）

3. 模型容器 Containers
 1. nn.Sequential(按顺序包装多个网络层) 
 2. nn.ModuleList(像list一样包装多个网络层)
 3. nn.ModuleDict（像dict一样包装多个网络层）
 

nn.Sequential: 按顺序包装网络层，将深度学习模型分为特征提取部分，全连接层为模型部分

1. 从下面看到，分为两部分第一部分features模型 特征提取
第二部分 classifier 用于分类
2. forward函数，只需要features处理输出，形状变换，然后classilier搞定
3. Sequential如何实现子模块拼接 (包括在forward中不同模块的合并)：
  1. 在Sequentia定义 的时候，会将每一层子模块的信息存入到_modules 这个有序字典中
  2. 前向传播的时候，Sequential的forward函数会便利字典，将 每一层拿出来做数据处理
  3. 串行实现，上一层的输出是下一层的输入：
    for module in self._module.values():
      input = module(input0
    return input



In [ ]:
 import troch.nn as nn
 class LeNetSequential(nn.Module):
     def __init__(self, classes):
         super(LeNetSequential, self).__init__()
         self.features = nn.Sequential(
             nn.Conv2d(3, 6, 5),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2),
             nn.Conv2d(6, 16, 5),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2),)
 
         self.classifier = nn.Sequential(
             nn.Linear(16*5*5, 120),
             nn.ReLU(),
             nn.Linear(120, 84),
             nn.ReLU(),
             nn.Linear(84, classes),)
 
     def forward(self, x):
         x = self.features(x)
         x = x.view(x.size()[0], -1)
         x = self.classifier(x)
         return x

通过对上面构建的网络层观察：子模块的字典键成了序号，如果网络层很多的话，很难通过对序号进行索引，所以可以对网络层进行命名，也就是：
 1. 在 nn.Sequential(OrderedDict({
   'cov1':nn.Cov2d(3,6,5)
 }))
 2. 有序的字典，字典中是网络名:网络层的形式(在代码运行的时候，container的init函数中就会判断这个网络是不是有序字典的形式，不是的话就按照序号编码)


nn.Sequential的总结：
 1. 顺序性：各网络层之间严格按照顺序构建，这时候一定要注意前后层数据的关系
 2. 自带 forward(): 自答的 forward 里，通过 for 循环依次执行前向传播运算

In [ ]:
class LeNetSequentialOrderDict(nn.Module):
     def __init__(self, classes):
         super(LeNetSequentialOrderDict, self).__init__()
 
         self.features = nn.Sequential(OrderedDict({
             'conv1': nn.Conv2d(3, 6, 5),
             'relu1': nn.ReLU(inplace=True),
             'pool1': nn.MaxPool2d(kernel_size=2, stride=2),
 
             'conv2': nn.Conv2d(6, 16, 5),
             'relu2': nn.ReLU(inplace=True),
             'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
         }))
 
         self.classifier = nn.Sequential(OrderedDict({
             'fc1': nn.Linear(16*5*5, 120),
             'relu3': nn.ReLU(),
 
             'fc2': nn.Linear(120, 84),
             'relu4': nn.ReLU(inplace=True),
 
             'fc3': nn.Linear(84, classes),
         }))
 
     def forward(self, x):
         x = self.features(x)
         x = x.view(x.size()[0], -1)
         x = self.classifier(x)
         return x

3.2 nn.ModuleList 包装一组网络容器，以迭代方式调用网络层
 1. append(): 在 ModuleList 后面添加网络层
 2. extend(): 拼接两个 ModuleList
 3. insert(): 指定在 ModuleList 中位置插入网络层
类似于我们的列表，只不过元素换成网络层


In [ ]:
class ModuleList(nn.Module):
    def __init__(self):
        super(ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(20)])

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x
# ModuleList 构建网络层就可以使用列表生成式构建，然后前向传播的时候也是遍历每一层，
# 进行计算即可

In [ ]:
3.3 nn.ModuleDict 包装一组网络层，以 索引的方式 调用网络层，主要方法
  1. clear(): 清空 ModuleDict
  2. items(): 返回可迭代的键值对(key-value pairs)
  3. keys(): 返回字典的键(key)
  4. values(): 返回字典的值(value)
  5. pop(): 返回一对键值对，并从字典中删除
# 从以下看出在选择网络层的时候比较实用，例如在做某些任务的时候
# 可以做对比

In [ ]:
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        self.choices = nn.ModuleDict({
            'conv': nn.Conv2d(10, 10, 3),
            'pool': nn.MaxPool2d(3)
        })

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'prelu': nn.PReLU()
        })

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x
    
net = ModuleDict()
fake_img = torch.randn((4, 10, 32, 32))
output = net(fake_img, 'conv', 'relu')    # 在这里可以选择我们的层进行组合
print(output)

# 前面通过self.choices这个 ModuleDict 可以选择卷积或者池化，
# 而下面通过self.activations这个 ModuleDict 可以选取是用哪个激活函数

1. nn.Sequential 顺序性 各网络层之间按照顺序执行，用于block构建
2. nn.ModuleList 迭代性 常用语大量重复网构建，for循环实现
3. nn.ModuleDict 用于可选择的网络层

In [ ]:

class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

Sequential 进行搭建的，分三部分，第一部分是一个 Sequential，由一系列的卷积池化模块构成，目的是提取图像的特征，然后是一个全局的池化层把特征进行整合，最后有一个 Sequential 是全连接层组成的，用于模型的分类。这样就完成了 AlexNet 网络的搭建，forward 函数汇总也是非常简单了